<a href="https://colab.research.google.com/github/jinkyukim-me/Summary-Seocho-Pytorch/blob/master/wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [0]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [3]:
wine = load_wine()
wine

{'DESCR': '.. _wine_dataset:\n\nWine recognition dataset\n------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 178 (50 in each of three classes)\n    :Number of Attributes: 13 numeric, predictive attributes and the class\n    :Attribute Information:\n \t\t- Alcohol\n \t\t- Malic acid\n \t\t- Ash\n\t\t- Alcalinity of ash  \n \t\t- Magnesium\n\t\t- Total phenols\n \t\t- Flavanoids\n \t\t- Nonflavanoid phenols\n \t\t- Proanthocyanins\n\t\t- Color intensity\n \t\t- Hue\n \t\t- OD280/OD315 of diluted wines\n \t\t- Proline\n\n    - class:\n            - class_0\n            - class_1\n            - class_2\n\t\t\n    :Summary Statistics:\n    \n    ============================= ==== ===== ======= =====\n                                   Min   Max   Mean     SD\n    ============================= ==== ===== ======= =====\n    Alcohol:                      11.0  14.8    13.0   0.8\n    Malic Acid:                   0.74  5.80    2.34  1.12\n    Ash:           

sklearn에서 제공하는 wine 에는 다음과 같은 필드가 있다.

* DESCR: 데이터 집합의 상세정보
* data: 와인 성분 데이터 (설명변수)
* feature_names: 와인 성분명
* target: 와인의 품종 데이터 (목적변수)
* target_names: 와인의 품종이름

In [4]:
pd.DataFrame(wine.data, columns=wine.feature_names)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.640000,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.380000,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.680000,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.800000,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.320000,1.04,2.93,735.0
5,14.20,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.750000,1.05,2.85,1450.0
6,14.39,1.87,2.45,14.6,96.0,2.50,2.52,0.30,1.98,5.250000,1.02,3.58,1290.0
7,14.06,2.15,2.61,17.6,121.0,2.60,2.51,0.31,1.25,5.050000,1.06,3.58,1295.0
8,14.83,1.64,2.17,14.0,97.0,2.80,2.98,0.29,1.98,5.200000,1.08,2.85,1045.0
9,13.86,1.35,2.27,16.0,98.0,2.98,3.15,0.22,1.85,7.220000,1.01,3.55,1045.0


In [5]:
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

다음은 목적 변수다. 0 부터 2까지의 값을 갖는 numpy배열이다.

일단 0과 1 분류만 해보도록 하자. 그리고, 각각의 데이터를 훈련데이터, 테스트 데이터로 나눈다.

In [0]:
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.2)

그리고 각각의 데이터를, 파이토치가 다룰 수 있는 형태로 정리한다.

In [0]:
# 데이터를 텐서 형태로 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).float()

그리고 설명변수와 목적변수의 텐서를 합친다. 이를 미니배치로 분할한다. 미니배치 사이즈는 16이다.

In [0]:
train = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train, batch_size=16, shuffle=True)

In [9]:
train[0]

(tensor([1.4380e+01, 1.8700e+00, 2.3800e+00, 1.2000e+01, 1.0200e+02, 3.3000e+00,
         3.6400e+00, 2.9000e-01, 2.9600e+00, 7.5000e+00, 1.2000e+00, 3.0000e+00,
         1.5470e+03]), tensor(0))

train 첫번째 데이터를 살짝 엿보면, 각각의 설명변수와 목적변수가 따로 담겨 있는 것을 알 수 있다.

신경망 구성

만들어볼 신경망 구성은 다음과 같다. 입력층, 중간층, 출력층이 각각 하나씩 있는 신경망을 구성한다. 입력층의 노드 개수는 13개 (설명변수가 13개 니깐)고, 중간층 노드의 개수는 96개 (내맘) 출력층 노드의 수는 2개 (목적변수가 0, 1 이니깐…) 다. 마지막 출력층에서는 0일 확률과 1일 확률을 뱉어낼 것이다.

In [0]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 입력층과 중간층 사의의 정의. 13개의 입력변수를 받아서 96개의 중간층 노드를 만든다.
        self.fc1 = nn.Linear(13, 96)
        # 중간층과 출력층 사의의 정의. 96개의 중간변수를 받아서 2개의 최종 노드를 만든다.
        self.fc2 = nn.Linear(96, 2)

    # 순전파
    def forward(self, x):
        # relu함수로 입력층을 변환한다.
        x = F.relu(self.fc1(x))
        # 이를 중간층으로 넘긴다.
        x = self.fc2(x)
        # 출력 함수로 log_softmax를 사용한다.
        return F.log_softmax(x)

# 선언
model = Net()

torch.nn.Module: 모든 신경망 모듈의 기본이 되는 클래스다. 이 안에 각 층, 함수, 신경망구조를 정의한다.

torch.nn.Linear: 입력데이터에 대해서 선형 변환
y=Ax+b
를 변환한다.

torch.nn.functional.relu: ReLU

torch.nn.fucntional.log_softmax: log softmax를 구현했다.

모형학습

In [11]:
# 오차함수. 분류 문제에는 교차 함수로 크로스 엔트로피를 사용한다고 했었다.
criterion = nn.CrossEntropyLoss()

# 최적화 담당. 경사 하강법을 적용하였다. 학습률은 0.01.
optimizer = optim.SGD(model.parameters(), lr=0.01)

#학습 300회 ㄱㄱ 씽
for epoch in range(300):
  # 누적 오차를 담당할 변수
  total_loss = 0
  
  # 아까 만들어놓은 미니 배치에서 각각 변수를 꺼내온다.
  for train_x, train_y in train_loader:
    # 각각의 값을 변수로 만든다.
    train_x, train_y = Variable(train_x), Variable(train_y)

    # 경사 초기화
    optimizer.zero_grad()

    # 순전파
    output = model(train_x)

    # 오차계산. ouutput 과 train_y 비교
    loss = criterion(output, train_y)

    # 역전파 계산
    loss.backward()

    # 가중치 업데이트. learning rate만큼 
    optimizer.step()

    #총 오차 업데이트
    total_loss += loss.data.item()

  # 10 회 마다 현재 오차를 출력
  if (epoch + 1) % 10 == 0:
    print(epoch + 1, total_loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


10 4.8476502895355225
20 4.806734204292297
30 4.780645132064819
40 4.757931351661682
50 4.813745081424713
60 4.781183362007141
70 4.764140725135803
80 4.7625685930252075
90 4.762501358985901
100 4.7436633706092834
110 4.781687915325165
120 4.762276113033295
130 4.707699716091156
140 4.800565719604492
150 4.8004220724105835
160 4.801235914230347
170 4.743316411972046
180 4.72355580329895
190 4.781462907791138
200 4.781197011470795
210 4.801648557186127
220 4.781902313232422
230 4.781840562820435
240 4.8024903535842896
250 4.741764783859253
260 4.801016926765442
270 4.823765993118286
280 4.742781281471252
290 4.762019872665405
300 4.78165066242218


정확도를 계산해보자

In [12]:
# 테스트 데이터를 집어 넣는다.
test_x, test_y = Variable(test_X), Variable(test_Y)
# 테스트 데이터를 집어 넣어서 학습 시킨 다음, max 값(확률이 더 높은 값)을 출력한다.
result = torch.max(model(test_x).data, 1)[1]
# 정확도 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
print("accuracy= ",accuracy)

accuracy=  0.4230769230769231


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
